In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "theano"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils


keras.backend.set_floatx('float64')

from keras.utils import np_utils


def set_reproducible():
    import tensorflow as tf
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '1960'
    rn.seed(1960)
    np.random.seed(1960)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K
    tf.set_random_seed(1960)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    # K.set_session(sess)

set_reproducible()


Using Theano backend.


# Build a Keras Model

In [2]:

def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation, Dropout
    from keras.utils import np_utils

    model = Sequential()
    model.add(Dense(5, input_shape=(4,) , activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model



In [3]:
iris  = datasets.load_iris()
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, train_size=0.8, test_size=0.2, random_state=1960)


In [4]:

from keras.wrappers.scikit_learn import KerasClassifier

clf = KerasClassifier(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(120, 4) (120,)


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
W0704 22:16:43.435081 140153694496576 blas.py:433] We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [5]:
print(clf.model.__dict__)

{'name': 'sequential_1', 'trainable': True, '_is_compiled': True, '_expects_training_arg': False, '_initial_weights': None, 'supports_masking': False, 'optimizer': <keras.optimizers.Adam object at 0x7f77995e0ac8>, '_updates': [], '_losses': [], '_per_input_losses': {}, '_per_input_updates': {}, '_layers': [<keras.engine.input_layer.InputLayer object at 0x7f77fa78a080>, <keras.layers.core.Dense object at 0x7f77fa78a908>, <keras.layers.core.Dropout object at 0x7f7799655438>, <keras.layers.core.Dense object at 0x7f779965df60>, <keras.layers.core.Activation object at 0x7f77fa78ac88>], '_outbound_nodes': [], '_inbound_nodes': [<keras.engine.base_layer.Node object at 0x7f77996685f8>], '_is_graph_network': True, '_uses_inputs_arg': True, 'outputs': [Softmax.0], 'inputs': [/dense_1_input], '_built': True, '_build_input_shape': None, '_compute_previous_mask': True, '_input_layers': [<keras.engine.input_layer.InputLayer object at 0x7f77fa78a080>], '_output_layers': [<keras.layers.core.Activation

In [6]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,4))
print(preds)


(30, 4)
[2]


# Generate SQL Code from the Model

In [7]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

def test_ws_sql_gen(pickle_data):
    WS_URL="https://sklearn2sql.herokuapp.com/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    # print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



In [8]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)

In [9]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."Feature_0" AS "Feature_0", "ADS"."Feature_1" AS "Feature_1", "ADS"."Feature_2" AS "Feature_2", "ADS"."Feature_3" AS "Feature_3" 
FROM "INPUT_DATA" AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."Feature_0" AS "Feature_0", keras_input."Feature_1" AS "Feature_1", keras_input."Feature_2" AS "Feature_2", keras_input."Feature_3" AS "Feature_3" 
FROM keras_input), 
layer_dense_1 AS 
(SELECT keras_input_1."KEY" AS "KEY", -0.0387618257795595 + 0.5681119286738643 * keras_input_1."Feature_0" + -0.24174715520925227 * keras_input_1."Feature_1" + -0.7698312687295721 * keras_input_1."Feature_2" + -0.41250696418645677 * keras_input_1."Feature_3" AS output_1, -0.033859804428020834 + -0.3486252182589338 * keras_input_1."Feature_0" + 0.5925973998496471 * keras_input_1."Feature_1" + -0.38658355527201277 * keras_input_1."Feature_2" + 0.5379057289261714 * keras_input_1."Feature_3" AS output_2, -0.04583993600484295 + 0.

# Execute the SQL Code

In [10]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(iris.data);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['TGT'] = iris.target
lTable['KEY'] = range(iris.data.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)

/home/antoine/.local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [11]:
sql_output = pd.read_sql(lSQL , conn);
sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)
conn.close()

In [12]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,None,None,None,0.003948,0.038439,0.957613,-5.534672,-3.258680,-0.043311,2,0.957613
74,74,None,None,None,0.031863,0.111751,0.856385,-3.446308,-2.191478,-0.155035,2,0.856385
9,9,None,None,None,0.217765,0.391007,0.391228,-1.524338,-0.939031,-0.938465,2,0.391228
88,88,None,None,None,0.047047,0.137924,0.815029,-3.056599,-1.981053,-0.204532,2,0.815029
25,25,None,None,None,0.202904,0.376683,0.420413,-1.595020,-0.976351,-0.866519,2,0.420413
5,5,None,None,None,0.209545,0.362378,0.428077,-1.562816,-1.015068,-0.848453,2,0.428077
48,48,None,None,None,0.222019,0.398795,0.379186,-1.504990,-0.919308,-0.969729,1,0.398795
117,117,None,None,None,0.002039,0.024921,0.973040,-6.195197,-3.692033,-0.027331,2,0.973040
83,83,None,None,None,0.007436,0.053985,0.938580,-4.901434,-2.919057,-0.063388,2,0.938580
105,105,None,None,None,0.000925,0.016779,0.982295,-6.985405,-4.087600,-0.017863,2,0.982295


# Keras Prediction

In [13]:
keras_output = pd.DataFrame()
keras_output_key = pd.DataFrame(list(range(iris.data.shape[0])), columns=['KEY']);
keras_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
keras_output_proba = pd.DataFrame(clf.predict_proba(iris.data), columns=['Proba_0', 'Proba_1', 'Proba_2'])
keras_output = pd.concat([keras_output_key, keras_output_score, keras_output_proba] , axis=1)
for class_label in [0, 1, 2]:
    keras_output['LogProba_' + str(class_label)] = np.log(keras_output_proba['Proba_' + str(class_label)])
keras_output['Decision'] = clf.predict(iris.data)
keras_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.003948,0.038439,0.957613,-5.534672,-3.258680,-0.043311,2
74,74,NaN,NaN,NaN,0.031863,0.111751,0.856385,-3.446308,-2.191478,-0.155035,2
9,9,NaN,NaN,NaN,0.217765,0.391007,0.391228,-1.524338,-0.939031,-0.938465,2
88,88,NaN,NaN,NaN,0.047047,0.137924,0.815029,-3.056599,-1.981053,-0.204532,2
25,25,NaN,NaN,NaN,0.202904,0.376683,0.420413,-1.595020,-0.976351,-0.866519,2
5,5,NaN,NaN,NaN,0.209545,0.362378,0.428077,-1.562816,-1.015068,-0.848453,2
48,48,NaN,NaN,NaN,0.222019,0.398795,0.379186,-1.504990,-0.919308,-0.969729,1
117,117,NaN,NaN,NaN,0.002039,0.024921,0.973040,-6.195197,-3.692033,-0.027331,2
83,83,NaN,NaN,NaN,0.007436,0.053985,0.938580,-4.901434,-2.919057,-0.063388,2
105,105,NaN,NaN,NaN,0.000925,0.016779,0.982295,-6.985405,-4.087600,-0.017863,2


# Comparing the SQL and Keras Predictions

In [14]:
sql_keras_join = keras_output.join(sql_output , how='left', on='KEY', lsuffix='_keras', rsuffix='_sql')


In [15]:
sql_keras_join.head(12)

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
0,0,NaN,NaN,NaN,0.225168,0.402124,0.372708,-1.490908,-0.910995,-0.986961,...,None,None,0.225168,0.402124,0.372708,-1.490908,-0.910995,-0.986961,1,0.402124
1,1,NaN,NaN,NaN,0.213670,0.396157,0.390173,-1.543322,-0.925946,-0.941164,...,None,None,0.213670,0.396157,0.390173,-1.543322,-0.925946,-0.941164,1,0.396157
2,2,NaN,NaN,NaN,0.231637,0.395776,0.372587,-1.462583,-0.926907,-0.987284,...,None,None,0.231637,0.395776,0.372587,-1.462583,-0.926907,-0.987284,1,0.395776
3,3,NaN,NaN,NaN,0.223402,0.367775,0.408823,-1.498782,-1.000283,-0.894474,...,None,None,0.223402,0.367775,0.408823,-1.498782,-1.000283,-0.894474,2,0.408823
4,4,NaN,NaN,NaN,0.232461,0.395807,0.371732,-1.459033,-0.926828,-0.989582,...,None,None,0.232461,0.395807,0.371732,-1.459033,-0.926828,-0.989582,1,0.395807
5,5,NaN,NaN,NaN,0.209545,0.362378,0.428077,-1.562816,-1.015068,-0.848453,...,None,None,0.209545,0.362378,0.428077,-1.562816,-1.015068,-0.848453,2,0.428077
6,6,NaN,NaN,NaN,0.234404,0.369327,0.396269,-1.450710,-0.996073,-0.925661,...,None,None,0.234404,0.369327,0.396269,-1.450710,-0.996073,-0.925661,2,0.396269
7,7,NaN,NaN,NaN,0.221153,0.385795,0.393052,-1.508902,-0.952449,-0.933813,...,None,None,0.221153,0.385795,0.393052,-1.508902,-0.952449,-0.933813,2,0.393052
8,8,NaN,NaN,NaN,0.226739,0.371122,0.402138,-1.483954,-0.991223,-0.910960,...,None,None,0.226739,0.371122,0.402138,-1.483954,-0.991223,-0.910960,2,0.402138
9,9,NaN,NaN,NaN,0.217765,0.391007,0.391228,-1.524338,-0.939031,-0.938465,...,None,None,0.217765,0.391007,0.391228,-1.524338,-0.939031,-0.938465,2,0.391228


In [16]:
condition = (sql_keras_join.Decision_sql != sql_keras_join.Decision_keras)
sql_keras_join[condition]

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
